In [ ]:
import pymysql as mariadb
import pickle
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer

In [ ]:
#establishing the dataframe
connection = mariadb.connect(host="localhost", database="masterarbeit",
                            user="USER", password="PASSWORD")

#filtering out the datasources where there are no comments or comments without date
sql = ("SELECT * "
       "FROM comment ")

df = pd.read_sql_query(sql, connection)

liste = pickle.load(open("lemmatized_kom.pkl", "rb"))

df["lemmatized"] = liste

In [ ]:
df['art_date'] = pd.to_datetime(df['art_date'])
df2 = df
df2 = df2.set_index(df2["art_date"])
df2 = df2[df2.index.month.isin([1,2,3,4,5,6])]
df2

In [ ]:
#vectorizer
vectorizer = TfidfVectorizer(max_features = 3000)
vecs = vectorizer.fit_transform(df['lemmatized'])

In [ ]:
km = KMeans(random_state=42)
visualizer = KElbowVisualizer(km, k=(2,25))
visualizer.fit(vecs)        
visualizer.show()   

In [ ]:
silhouette_score = []
kmeansvalues = []
for i in range(2,25):
    km = KMeans(n_clusters=i, random_state=42)

    km.fit_predict(vecs)

    # Calculate Silhoutte Score

    score = metrics.silhouette_score(vecs, km.labels_, metric='euclidean')
    silhouette_score.append(score)
    kmeansvalues.append(i)
    print(i, 'Silhouetter Score: %.10f' % score)

In [ ]:
plt.rcParams["figure.figsize"] = (14,9)
plt.title("Silhouette Score per k Cluster")
#plt.axhline(y=0.0, color='black', linestyle='--')
plt.plot(kmeansvalues, [x for x in silhouette_score], marker = "o")

In [ ]:
X = vecs.toarray()

In [ ]:
score = []
for x in range(2,25):
    kmeans_model = KMeans(n_clusters=x, random_state=42).fit(vecs)
    labels = kmeans_model.labels_
    m = metrics.calinski_harabasz_score(X, labels)
    score.append(m)
    print(x, m)

In [ ]:
plt.plot([x for x in range(2,25)], score, marker = "o")
plt.title("Calinski Harabasz Score")

In [ ]:
scores = []
for x in range(2,25):
    kmeans_model = KMeans(n_clusters=x, random_state=42).fit(vecs)
    labels = kmeans_model.labels_
    m = metrics.davies_bouldin_score(X, labels)
    scores.append(m)
    print(x, m)

In [ ]:
plt.plot([x for x in range(2,25)], scores, marker = "o")
plt.title("Davies Bouldin Score")